In [1]:
import numpy
import sklearn
import survwrap

In [2]:
X, y = survwrap.load_test_data()
X.shape, y.shape

((198, 84), (198,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [3]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [4]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y,
    stratify=survwrap.get_indicator(y), 
    random_state=2308,
)

In [5]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((148, 84), (50, 84))

In [7]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [108]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(51, 38, 13)

# test coxnet

In [6]:
coxnet = survwrap.CoxNet(rng_seed=2308)
coxnet.fit(X_train, y_train)

CoxNet(rng_seed=2308, l1_ratio=0.5, verbose=False, fit_baseline_model=False)

In [7]:
coxnet.score(X_test, y_test)

0.5330578512396694

In [8]:
from survwrap.hyper import optimize

In [9]:
r1 = optimize(survwrap.CoxNet(), X_train, y_train, mode='sklearn-grid')
r1

{'l1_ratio': FloatParameter(min=-4.59511985013459, max=4.595119850134584, num_steps=7, transform=<function make_all_params.<locals>.<lambda> at 0x7fd5ea3f65e0>)}
{'l1_ratio': [0.010000000000000002, 0.04464190557436299, 0.17774412460002154, 0.49999999999999933, 0.8222558753999777, 0.9553580944256369, 0.99]}
7


(CoxNet(rng_seed=-1, l1_ratio=0.49999999999999933, verbose=False, fit_baseline_model=False),
 {'l1_ratio': 0.49999999999999933},
 GridSearchCV(estimator=CoxNet(),
              param_grid={'l1_ratio': [0.010000000000000002, 0.04464190557436299,
                                       0.17774412460002154, 0.49999999999999933,
                                       0.8222558753999777, 0.9553580944256369,
                                       0.99]}))

In [11]:
r1[0].score(X_test, y_test)

0.5

In [12]:
r2 = optimize(survwrap.CoxNet(), X_train, y_train, mode='optuna', tries=10)
r2

[I 2023-09-07 13:23:10,887] A new study created in memory with name: prova


{'l1_ratio': FloatParameter(min=-4.59511985013459, max=4.595119850134584, num_steps=7, transform=<function make_all_params.<locals>.<lambda> at 0x7fd5ea3f65e0>)}
{'l1_ratio': [0.010000000000000002, 0.04464190557436299, 0.17774412460002154, 0.49999999999999933, 0.8222558753999777, 0.9553580944256369, 0.99]}
7
trial params: {'l1_ratio': 0.610308175389875}


[I 2023-09-07 13:23:15,002] Trial 0 finished with value: 0.5756680088279384 and parameters: {'l1_ratio': 0.44860780170214287}. Best is trial 0 with value: 0.5756680088279384.


score 0.5756680088279384 [0.52272727 0.66666667 0.70857143 0.56730769 0.56818182]
trial params: {'l1_ratio': 0.8784295582382612}


[I 2023-09-07 13:23:19,826] Trial 1 finished with value: 0.5696871027673664 and parameters: {'l1_ratio': 1.9776418579115749}. Best is trial 0 with value: 0.5756680088279384.


...

score 0.5709505042232558 [0.52840909 0.640625   0.73714286 0.55769231 0.56060606]
trial params: {'l1_ratio': 0.986090442853965}


[I 2023-09-07 13:23:49,337] Trial 8 finished with value: 0.5747258775871312 and parameters: {'l1_ratio': 4.261171909092954}. Best is trial 3 with value: 0.5768626343010168.


score 0.5747258775871312 [0.53409091 0.625      0.73142857 0.5625     0.57575758]
trial params: {'l1_ratio': 0.2551748719138359}


[I 2023-09-07 13:23:52,973] Trial 9 finished with value: 0.5739527398174206 and parameters: {'l1_ratio': -1.0712003810904176}. Best is trial 3 with value: 0.5768626343010168.


score 0.5739527398174206 [0.51136364 0.671875   0.72571429 0.56730769 0.56818182]


In [ ]:
r2